In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')
import pandas as pd

os.environ['PATH'] = '/Users/ravi.tej/anaconda3/envs/bertopicenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin'

In [3]:
import boto3
from botocore.config import Config
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


In [4]:
my_config = Config(
    region_name='ap-south-1',
    retries={
        'max_attempts': 5,
        'mode': 'standard'
    },
    max_pool_connections=40  # Increase the pool size
)

# Create a SageMaker Runtime client with the custom configuration
sess1 = boto3.session.Session()
sagemaker_runtime = sess1.client("sagemaker-runtime", config=my_config)

In [5]:
df = pd.read_csv('gpt-4-cleaned_responses.csv')

In [42]:
df['output_tokens'].describe()

count    1122.000000
mean      359.662210
std       160.797769
min        11.000000
25%       310.000000
50%       401.000000
75%       472.000000
max       733.000000
Name: output_tokens, dtype: float64

In [40]:
# from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('teknium/OpenHermes-2.5-Mistral-7B', trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
SYSTEM_PROMPT = df.iloc[0].system_prompt

In [7]:
SYSTEM_PROMPT

"As the chief editor for an Indian finance and business portal, your role involves providing accurate and relevant responses to inquiries that fall within the scope of finance, business, and key entities such as India, RBI, Reliance, the Fed, etc. You use the articles that you have found to answer these queries. It is crucial to ensure that your answers are tailored to the query's requirements, employing only one of the following structured formats based on the query:\n1. **Quick Insights**: Provide concise, direct answers spanning a few lines. No need for any headline - directly give the answer and don't mention 'Quick Insights' in the answer. Ideal for straightforward questions seeking immediate information.\n2. **Step-by-Step Guide**: Offer a detailed, numbered guide with clear labels for each step. Start with a contextual headline, include a brief introduction and conclusion to frame the context, catering to inquiries that necessitate a methodical approach.\n3. **Deep Dive**: Const

In [43]:
content = df.iloc[312].user_prompt

In [44]:
content

' || user_query: retirement  || today date: 19 January 2024 || 1: {\'title\': Top 5 Retirement Mutual Funds to Invest in 2023, \'published_date\': 24 May 2022, \'content\': The goal of retirement planning is to figure out what kind of income you might need in retirement and then narrow down the options for getting there. Mutual funds make the cut as far as investment opportunities for financial goals go because saving for retirement is primarily a category that involves long-term planning.  Nevertheless, in order to build up the preferred income, you might not want to invest in any accessible mutual fund category. The finest mutual fund investments for retirement savings are those that provide diversification benefits while lowering uncertainties.  Have you been thinking of retirement options? If that\'s the case, you\'ve come to the right place. Retirement planning is critical, and it\'s especially important to get started early and correctly. You might be wondering, "Why do I need a 

In [45]:
messages = [{"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"{content}\n"}]
context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))

In [46]:
params = {
    "do_sample": True,
    "top_p": 0.95,
    "temperature": 0.5,
    "max_new_tokens": 1024,
    "repetition_penalty": 1.02,
    "stop": ["###", "</s>", '<|im_end|>']
}
request = {"inputs": context_prompt, "parameters": params, "stream": False}
request['parameters']['adapter_id'] = 'lorax/OpenHermes-2.5-Adapter-Search-ViolinAvocadoNebula'
request['parameters']['adapter_source'] = 's3'

In [47]:
import time

In [48]:
t = time.time()
import json
response = sagemaker_runtime.invoke_endpoint(
            EndpointName='LORAX-Openhermes',
            Body=json.dumps(request),
            ContentType="application/json")
print(f'done in {time.time() - t}')

done in 29.017647981643677


In [49]:
k = response['Body'].read()

In [50]:
json.loads(json.loads(k)[0]['generated_text'])

'"**Step-by-Step Guide to Effective Retirement Planning**<br><br>Planning for retirement is crucial for ensuring financial security and peace of mind in your golden years. Here\'s a structured approach to help you navigate the process effectively.<br><br>**Introduction:**<br>Retirement planning involves assessing your current financial situation, setting retirement goals, and choosing the right investment options to fund your post-retirement life.<br><br>**Step 1: Determine Your Retirement Age**<br>Choose the age at which you wish to retire. This will help you estimate the time you have to accumulate your retirement fund.<br><br>**Step 2: Calculate Your Retirement Corpus**<br>Consider your current expenses and project them into your retirement years, accounting for inflation. This will give you an idea of the size of the retirement fund you need to maintain your desired lifestyle.<br><br>**Step 3: Evaluate Your Current Financial Situation**<br>Analyze your income, assets, liabilities, 

In [36]:
len(tokenizer.encode(''"**Step-by-Step Guide: Planning for a Dream Wedding**<br><br>Planning for a dream wedding involves a mix of financial planning, investment strategies, and practical considerations. Here\'s a structured guide to help you navigate through the process:<br><br>1. **Start Early with Savings**: Begin saving for your wedding as soon as possible. The earlier you start, the more time your investments have to grow, potentially increasing your wedding fund significantly.<br><br>2. **Invest in Various Financial Products**: Consider investing in a mix of equity mutual funds, debt mutual funds, and gold ETFs. Allocate a portion of your savings to equity for higher returns over a longer period. Use debt funds for shorter-term savings goals and gold ETFs for the security and lower cost compared to physical gold.<br><br>3. **Establish a Systematic Investment Plan**: Set up a regular investment schedule where you contribute a fixed amount to your mutual fund portfolio. This disciplined approach takes advantage of cost averaging, allowing you to buy more units when prices are low and fewer units when prices are high.<br><br>4. **Evaluate and Adjust Your Investments**: Regularly review the performance of your mutual funds and assess if they are meeting your expectations. If necessary, consider adjusting your investment allocation or switching to different funds that better align with your goals. However, avoid making impulsive decisions based on short-term market fluctuations.<br><br>5. **Use Wedding Loans for Additional Funding**: If your investments fall short of covering all wedding expenses, consider applying for a marriage loan. These loans can cover the remaining costs and are designed to help you fulfill your wedding dreams without depleting your savings entirely.<br><br>By following this guide and taking a strategic approach to financing your wedding, you can ensure that your special day is everything you have dreamed of. Remember, investing in your future requires careful planning and discipline."''))

417